In [4]:
#Dining Philosopher 
import threading 
import random 
import time

class Philosopher(threading.Thread): 
    running = True

    def __init__(self, index, forkOnLeft, forkOnRight): 
        threading.Thread.__init__(self)
        self.index = index 
        self.forkOnLeft = forkOnLeft 
        self.forkOnRight = forkOnRight

    def run(self): 
        while(self.running):
            time.sleep(30)
            print(f'Philosopher {self.index} is hungry.') 
            self.dine()

    def dine(self):
        fork1, fork2 = self.forkOnLeft, self.forkOnRight
        while self.running: 
            fork1.acquire()
            locked = fork2.acquire(False) 
            if locked: break 
            fork1.release()
            print(f'Philosopher {self.index} swaps forks.') 
            fork1, fork2 = fork2, fork1
        else:
            return 
        self.dining() 
        fork2.release() 
        fork1.release()

    def dining(self):
        print(f'Philosopher {self.index} starts eating.') 
        time.sleep(30)
        print(f'Philosopher {self.index} finishes eating and leaves to think.') 

def main():
    forks = [threading.Semaphore() for n in range(5)] 
    philosophers = [Philosopher(i, forks[i%5], forks[(i+1)%5]) for i in range(5)] 
    Philosopher.running = True 
    for p in philosophers: 
        p.start() 
    time.sleep(100) 
    Philosopher.running = False 
    print("Now we are finishing.")

if __name__ == "__main__":
    main()


Philosopher 2 finishes eating and leaves to think.Philosopher 1 is hungry.

Philosopher 3 starts eating.
Philosopher 0 finishes eating and leaves to think.
Philosopher 4 is hungry.
Philosopher 2 is hungry.Philosopher 4 is hungry.
Philosopher 4 starts eating.
Philosopher 3 is hungry.
Philosopher 3 swaps forks.
Philosopher 1 is hungry.
Philosopher 1 starts eating.
Philosopher 0 is hungry.

Philosopher 3 finishes eating and leaves to think.
Philosopher 1 finishes eating and leaves to think.Philosopher 4 finishes eating and leaves to think.
Philosopher 3 starts eating.

Philosopher 0 starts eating.
Philosopher 2 swaps forks.
Philosopher 1 is hungry.Philosopher 4 is hungry.

Philosopher 3 finishes eating and leaves to think.
Philosopher 0 finishes eating and leaves to think.
Philosopher 4 starts eating.
Philosopher 1 starts eating.
Philosopher 2 swaps forks.
Now we are finishing.
Philosopher 1 finishes eating and leaves to think.Philosopher 4 finishes eating and leaves to think.
Philosopher

In [7]:
#Reader-Writer
import threading as thread 
import random

global x 
x = 0
lock = thread.Lock() 

def Reader():
    global x
    print('\nReader is Reading!') 
    lock.acquire() 
    print(f'\nShared Data:{x}') 
    lock.release()
    print() 

def Writer():
    global x
    print('\nWriter is Writing!') 
    lock.acquire()
    x += 1
    print('Writer is Releasing the lock!') 
    lock.release()
    print()

if __name__ == '__main__': 
    for i in range(0, 10):
        randomNumber = random.randint(0, 100) 
        if(randomNumber > 50):
            t1 = thread.Thread(target=Reader) 
            t1.start()
        else:
            t2 = thread.Thread(target=Writer) 
            t2.start()
    t1.join()
    t2.join()




Writer is Writing!
Writer is Releasing the lock!


Reader is Reading!

Shared Data:1


Writer is Writing!
Writer is Releasing the lock!


Reader is Reading!

Shared Data:2


Writer is Writing!
Writer is Releasing the lock!


Writer is Writing!
Writer is Releasing the lock!


Reader is Reading!

Shared Data:4


Writer is Writing!
Writer is Releasing the lock!


Reader is Reading!

Shared Data:5


Reader is Reading!

Shared Data:5



In [10]:
# producer-consumer problem

import threading 
import time 
items=[]
items_cv=threading.Condition() 

def prod():
    print("I'm the producer") 
    for i in range(30):
        with items_cv: 
            items.append(i) 
            items_cv.notify()
        time.sleep(1) 

def consumer():
    print("I'm a consumer",threading.current_thread().name) 
    while True:
        with items_cv: 
            while not items:
                items_cv.wait() 
            x = items.pop(0)
            print(threading.current_thread().name,"got",x) 
        time.sleep(5)

con=[threading.Thread(target=consumer) for i in range(10)]
for co in con: 
    co.daemon=True 
    co.start()

prod()



I'm a consumer Thread-45 (consumer)
I'm a consumer Thread-46 (consumer)
I'm a consumer Thread-47 (consumer)
I'm a consumer Thread-48 (consumer)
I'm a consumer Thread-49 (consumer)
I'm a consumer Thread-50 (consumer)
I'm a consumer Thread-51 (consumer)
I'm a consumer Thread-52 (consumer)
I'm a consumer Thread-53 (consumer)
I'm a consumer Thread-54 (consumer)
I'm the producer
Thread-45 (consumer) got 0
Thread-46 (consumer) got 1
Thread-47 (consumer) got 2
Thread-48 (consumer) got 3
Thread-49 (consumer) got 4
Thread-50 (consumer) got 5
Thread-51 (consumer) got 6
Thread-52 (consumer) got 7
Thread-53 (consumer) got 8
Thread-54 (consumer) got 9
Thread-45 (consumer) got 10
Thread-46 (consumer) got 11
Thread-47 (consumer) got 12
Thread-48 (consumer) got 13
Thread-49 (consumer) got 14
Thread-50 (consumer) got 15
Thread-51 (consumer) got 16
Thread-52 (consumer) got 17
Thread-53 (consumer) got 18
Thread-54 (consumer) got 19
Thread-45 (consumer) got 20
Thread-46 (consumer) got 21
Thread-47 (consum

In [12]:
#semaphores – Full and Empty

import threading 
import time 

Cap = 10
buf = [-1 for i in range(Cap)] 
inind = 0
outind = 0
mutex = threading.Semaphore() 
empty = threading.Semaphore(Cap) 
full = threading.Semaphore(0)

class Producer(threading.Thread): 
    def run(self):
        global Cap, buf, inind, outind 
        global mutex, empty, full 
        items_produced = 0
        counter = 0
        while items_produced < 20: 
            empty.acquire() 
            mutex.acquire()
            counter += 1 
            buf[inind] = counter
            inind = (inind + 1)%Cap 
            print(f"Producer produced:{counter}") 
            mutex.release()
            full.release() 
            time.sleep(1) 
            items_produced += 1

class Consumer(threading.Thread): 
    def run(self):
        global Cap, buf, inind, outind, counter 
        global mutex, empty, full 
        items_consumed = 0
        while items_consumed < 20: 
            full.acquire() 
            mutex.acquire()
            item = buf[outind]
            outind = (outind + 1)%Cap
            print(f"Consumer consumed item:{item}") 
            mutex.release()
            empty.release() 
            time.sleep(2.5) 
            items_consumed += 1

producer = Producer() 
consumer = Consumer() 
consumer.start() 
producer.start() 
producer.join() 
consumer.join()


Producer produced:1
Consumer consumed item:1
Producer produced:2
Producer produced:3
Consumer consumed item:2
Producer produced:4
Producer produced:5
Consumer consumed item:3
Producer produced:6
Producer produced:7
Producer produced:8
Consumer consumed item:4
Producer produced:9
Producer produced:10
Consumer consumed item:5
Producer produced:11
Producer produced:12
Producer produced:13
Consumer consumed item:6
Producer produced:14
Producer produced:15
Consumer consumed item:7
Producer produced:16
Producer produced:17
Consumer consumed item:8
Producer produced:18
Consumer consumed item:9
Producer produced:19
Consumer consumed item:10
Producer produced:20
Consumer consumed item:11
Consumer consumed item:12
Consumer consumed item:13
Consumer consumed item:14
Consumer consumed item:15
Consumer consumed item:16
Consumer consumed item:17
Consumer consumed item:18
Consumer consumed item:19
Consumer consumed item:20
